### Setup libraries

In [5]:
# pip install annoy # conda install -c conda-forge python-annoy
# pip install compress_fasttext
# pip install python-telegram-bot --upgrade
# pip install pymorphy2
# pip install stop_words

### Import libraries

In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:72.5% !important; }</style>"))

In [2]:
import numpy as np
import pandas as pd

import io
import re
import time
import unicodedata
from pathlib import Path
from linecache import getline
from tqdm.notebook import tqdm

import annoy
import string
from pymorphy2 import MorphAnalyzer
from stop_words import get_stop_words
import compress_fasttext
from gensim.models import FastText, KeyedVectors
from gensim.models.fasttext import FastTextKeyedVectors

import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.layers import Input, Dense
import tensorflow_datasets as tfds

from telegram.ext import Updater, CommandHandler, MessageHandler, Filters

import matplotlib.pyplot as plt

In [3]:
plt.rcParams.update({'font.size': 14})
pd.set_option('precision', 3)
pd.set_option('max_columns', 100)
pd.set_option('display.float_format', lambda x: '%.5f' % x)
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 500)
pd.set_option('max_colwidth', 300)

In [4]:
# device_name = tf.test.gpu_device_name()
# if device_name != '/device:GPU:0':
#     raise SystemError('GPU device not found')
# print('Found GPU at: {}'.format(device_name))

### Paths to directories and files

In [5]:
# from google.colab import drive
# drive.mount('/content/gdrive')

In [6]:
DRAFT_DATASET_PATH = 'D:/kaggle/mailru/Otvety.txt'
# DRAFT_DATASET_PATH = '/content/gdrive/MyDrive/Colab Notebooks/my_projects/Otvety.txt'

### Loading data

In [8]:
lines = io.open(DRAFT_DATASET_PATH, encoding='UTF-8').read()

### Preprocessing data

In [26]:
def preprocess_text(w):
    w = re.sub(':\)', '', w)
    w = re.sub('[)"]', '', w)
    w = re.sub('<[^>]+>', ' ', w)
    
    w = re.sub('\s*\?\s*\.', '?', w)
    w = re.sub('\s*\!\s*\.', '!', w)
    w = re.sub('\s*\.', '.', w)
    w = re.sub('\.+', '.', w)
    
    w = re.sub('---', 'QUESTION', w)
    
    return w

In [27]:
# print(type(lines))
print(len(lines))

1072593541


In [9]:
NUM_EXAMPLES = 12500000
text = preprocess_text(str(lines))
print(text[:1000])

In [ ]:
print(len(text))

In [ ]:
morpher = MorphAnalyzer()
sw = set(get_stop_words('ru'))

In [ ]:
def split_by_sentence(sent):
    sent = text.split('\nQUESTION\n')[1:]
    
    question = []
    answer = []

    for se in sent:
        se = se.split('\n')
        se = [i for i in se if (i not in string.punctuation)]
        se = [morpher.parse(i.lower())[0].normal_form for i in se]
        se = [i for i in se if i not in sw and i != '']

        question.append(se[0].strip())
        answer.append(' '.join([f' {s}' for s in se[1:]]))
    
    return question, answer

In [ ]:
question, answer = split_by_sentence(text)
question[0], answer[0]

In [ ]:
# df = pd.DataFrame({'questions':question, 'answers':answer})

In [ ]:
# df.head()

In [17]:
# for w in df['questions']:
#     re.sub('[^a-zA-Zа-яА-Я,.!?]+', ' ', w)

# for w in df['answers']:
#     re.sub('[^a-zA-Zа-яА-Я,.!?]+', ' ', w)

### FastText Model

In [16]:
core_numbers = 4
ft_width = 300

In [17]:
flag_ft = False

ft_file = Path('cc.ru.300.bin')
if ft_file.is_file():
    model_ft_wv = FastTextKeyedVectors.load('cc.ru.300.bin')
    flag_ft = True

### Evaluate

In [10]:
# text_generator('Что такое ЭБУ?')

In [11]:
# text_generator('Чем заняться?')

In [12]:
# text_generator('Что приготовить?')

In [ ]:
# text_generator('Что посмотреть?')

In [ ]:
# text_generator('Что послушать?')

In [ ]:
# text_generator('Что почитать?')

In [ ]:
# text_generator('Как подключить интернет?')

In [13]:
# text_generator('Как починить машину?')

### python-telegram-bot

In [114]:
updater = Updater(token='')
dispatcher = updater.dispatcher

In [117]:
def answer(update, context):
    context.bot.send_message(chat_id=update.effective_chat.id, text=text_generator(update.message.text))
    
answer_handler = MessageHandler(Filters.text, answer)
dispatcher.add_handler(answer_handler)

In [118]:
chat = True

if chat:
    updater.start_polling()
    print('@MyPersonalAssistanBot - online')
else:
    updater.stop()
    print('@MyPersonalAssistanBot - offline')

@MyPersonalAssistanBot - online
